In [22]:
import os
import tensorflow as tf
import random
import shutil
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adadelta, Adagrad
print(tf.__version__)

2.1.0


In [2]:
animal_ears_SAMPLE_DIR = 'sample/animal_ears'
human_ears_SAMPLE_DIR = 'sample/human_ears'

animal_earss = os.listdir(animal_ears_SAMPLE_DIR)
human_earss = os.listdir(human_ears_SAMPLE_DIR)

In [3]:
training_size = 90

animal_earss_train = random.sample(animal_earss, len(animal_earss)*training_size//100)
human_earss_train = random.sample(human_earss, len(human_earss)*training_size//100)

animal_earss_test = [item for item in animal_earss if item not in animal_earss_train]
human_earss_test = [item for item in human_earss if item not in human_earss_train]

In [7]:
BASE_DIR = 'image'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VAL_DIR = os.path.join(BASE_DIR, 'val')

animal_ears_TRAIN_DIR = os.path.join(TRAIN_DIR, 'animal_ears')
human_ears_TRAIN_DIR = os.path.join(TRAIN_DIR, 'human_ears')
animal_ears_VAL_DIR = os.path.join(VAL_DIR, 'animal_ears')
human_ears_VAL_DIR = os.path.join(VAL_DIR, 'human_ears')

In [5]:
for image in tqdm(animal_earss_train):
    src = os.path.join(animal_ears_SAMPLE_DIR, image)
    des = animal_ears_TRAIN_DIR
    shutil.copy(src, des)

for image in tqdm(human_earss_train):
    src = os.path.join(human_ears_SAMPLE_DIR, image)
    des = human_ears_TRAIN_DIR
    shutil.copy(src, des)

for image in tqdm(animal_earss_test):
    src = os.path.join(animal_ears_SAMPLE_DIR, image)
    des = animal_ears_VAL_DIR
    shutil.copy(src, des)

for image in tqdm(human_earss_test):
    src = os.path.join(human_ears_SAMPLE_DIR, image)
    des = human_ears_VAL_DIR
    shutil.copy(src, des)

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 608.25it/s]


In [32]:
train_datagen = ImageDataGenerator(
    rescale = 1.255,
    rotation_range = 30,
    height_shift_range = 0.4,
    width_shift_range = 0.4,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'
)
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    batch_size = 32,
    target_size= (54, 54),
    class_mode = 'binary'
)

val_datagen =  ImageDataGenerator(rescale = 1.255)
val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    batch_size = 32,
    target_size= (54, 54),
    class_mode = 'binary'
)

Found 3600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [46]:
model = tf.keras.Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(54, 54, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation=tf.nn.leaky_relu))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 52, 52, 16)        448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 16)        2320      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               295040    
_________________________________________________________________
dropout (Dropout)            (None, 128)              

In [49]:
model.compile(optimizer=Adam(lr=0.1), loss='binary_crossentropy', metrics=['accuracy'])

In [50]:
history = model.fit(
    train_generator,
    epochs = 10,
    validation_data = val_generator,
    verbose = 1
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 113 steps, validate for 13 steps
Epoch 1/10
113/113 [==============================] - 16s 142ms/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5000
Epoch 2/10
113/113 [==============================] - 18s 157ms/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5000
Epoch 3/10
113/113 [==============================] - 17s 148ms/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5000
Epoch 4/10
113/113 [==============================] - 19s 167ms/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5000
Epoch 5/10
113/113 [==============================] - 17s 146ms/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5000
Epoch 6/10
113/113 [==============================] - 17s 147ms/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5000
Epoch 7/10
113/113 [==============================] - 17s 148ms/step - lo

In [42]:
tf.keras.backend.clear_session()